In [3]:
from __future__ import annotations

from datasets import load_dataset

/opt/homebrew/anaconda3/envs/mteb/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
ds = load_dataset("agkphysics/AudioSet", "balanced")
ds

DatasetDict({
    train: Dataset({
        features: ['video_id', 'audio', 'labels', 'human_labels'],
        num_rows: 18685
    })
    test: Dataset({
        features: ['video_id', 'audio', 'labels', 'human_labels'],
        num_rows: 17142
    })
})

In [6]:
from collections import defaultdict

import numpy as np


def undersample_data_indices(y, samples_per_label, idxs=None):
    """Undersample data to have samples_per_label samples of each label"""
    sample_indices = []
    if idxs is None:
        idxs = np.arange(len(y))
    np.random.shuffle(idxs)
    label_counter = defaultdict(int)
    for i in idxs:
        if any((label_counter[label] < samples_per_label) for label in y[i]):
            sample_indices.append(i)
            for label in y[i]:
                label_counter[label] += 1
    return sample_indices, idxs

In [ ]:
sample_indices, _ = undersample_data_indices(ds["train"]["labels"], 8, None)
len(sample_indices)

# 16: 6018
# 8: 3010

3010

In [ ]:
test_sample_indices, _ = undersample_data_indices(ds["test"]["labels"], 6, None)
len(test_sample_indices)

# 4: 1556
# 6: 2153

2153

In [21]:
from datasets import DatasetDict

ds1 = DatasetDict(
    {
        "train": ds["train"].select(sample_indices),
        "test": ds["test"].select(test_sample_indices),
    }
)
ds1

DatasetDict({
    train: Dataset({
        features: ['video_id', 'audio', 'labels', 'human_labels'],
        num_rows: 3010
    })
    test: Dataset({
        features: ['video_id', 'audio', 'labels', 'human_labels'],
        num_rows: 2153
    })
})

In [ ]:
from huggingface_hub import create_repo

repo_name = "mteb/audioset"

WRITE_TOK = ""
create_repo(repo_name, repo_type="dataset", token=WRITE_TOK)

ds1.push_to_hub(repo_name, token=WRITE_TOK)

Uploading the dataset shards: 100%|██████████| 6/6 [01:47<00:00, 17.96s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/mteb/audioset/commit/168a7e681ee40609129535d49855c7e3e77e5efa', commit_message='Upload dataset', commit_description='', oid='168a7e681ee40609129535d49855c7e3e77e5efa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mteb/audioset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mteb/audioset'), pr_revision=None, pr_num=None)